In [2]:
!pip install SALib tqdm joblib
from joblib import Parallel, delayed
import numpy as np
from scipy.integrate import odeint
from SALib.sample import sobol
from tqdm import tqdm
import scipy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for SALib from https://files.pythonhosted.org/packages/e1/40/393b381779d379afbb0e281d9f69cb511022e41a726f7871a929faec2b11/salib-1.5.1-py3-none-any.whl.metadata
  Using cached salib-1.5.1-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/b2/07/8cbb75d6cfbe8712d8f7f6a5615f083c6e710ab916b748fbb20373ddb142/multiprocess-0.70.17-py311-none-any.whl.metadata
  Using cached multiprocess-0.70.17-py311-none-any.whl.metadata (7.2 kB)
  Obtaining dependency information for dill>=0.3.9 from https://files.pythonhosted.org/packages/46/d1/e73b6ad76f0b1fb7f23c35c6d95dbc506a9c8804f43dda8cb5b0fa6331fd/dill-0.3.9-py3-none-any.whl.metadata
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
Using cached salib-1.5.1-py3-none-any.whl (778 kB)
Using cached multiprocess-0.70.17-py311-none-any.whl (144 kB)
Using c


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def enzymic_amox(t,y, 
kcat1,
kcat2,
Km1,
Km2,  
Tmax, 
Ken,  
kAB,  
kAN,  
kAOH, 
kNH):
    FAB = 0
    FNH = 0 
    
    CAB = y[0]
    CAN = y[1]
    CNH = y[2]
    CAOH = y[3]

    Cez = 1

    # Consumo de ester
    VAB = (kcat1*CAB*Cez)/((Km1*(1 + (CAN/kAN) + (CAOH/kAOH))) + CAB)
    
    # Hidrolise de amoxicilina
    VAN = (kcat2*CAN*Cez)/((Km2*(1 + (CAB/kAB) + (CNH/kNH) + (CAOH/kAOH))) + CAN)
    
    # Enzima saturada com 6-apa
    X   = CNH/(Ken + CNH)
    
    # Sintese enzimatica
    VS  = VAB*Tmax*X

    # Hidrolise de ester
    Vh1 = (VAB - VS) 

    dy = np.zeros(4)

    # C. ester
    dy[0] = ((-(VS - VAN) - (Vh1 + VAN)) + FAB) 
    
    # C. amox
    dy[1] = (VS - VAN)                         
    
    # C. 6-apa
    dy[2] = (-(VS - VAN) + FNH)                
    
    # C. POHPG
    dy[3] =  (Vh1 + VAN)
    
    return np.array(dy)      

In [3]:
kcat1        = 0.178 #Constante catalítica do consumo do éster (mmol/i.u. per min)
 
kcat2        = 0.327 #Constante catalítica da hidrólise da amoxicilina (mmol/i.u. per min)
 
Km1          = 7.905 #Constante de Michaelis-Menten ou constante de afinidade para consumo do éster(mM) 
 
Km2          = 12.509 #Constante de Michaelis-Menten ou constante de afinidade para hidrólise da amoxicilina(mM)
 
Tmax         = 0.606 #Taxa de conversão máxima do complexo acil-enzima-núcleo em produto
 
Ken          = 14.350 #Constante de adsorção do 6-APA
 
kAB          = 3.78 #Constante de inibição do éster (POHPGME)(mM)
 
kAN          = 9.174 #Constante de inibição da amoxicilina (mM)
 
kAOH         = 10.907 #Constante de inibição do POHPG, produto da hidr�lise da amoxicilina (mM)
 
kNH          = 62.044 #Constante de inibição do 6-APA

P = [kcat1,kcat2,Km1,Km2,Tmax,Ken,kAB,kAN,kAOH,kNH]

In [4]:
def ode15s_amox(P, CI, t):
    return scipy.integrate.solve_ivp(enzymic_amox,t_span=(t[0],t[-1]),t_eval=t,y0=CI,method='RK45',args=P).y.T

In [16]:
inital_conditions = {
    '80_60':[80.0,0.0,60.0,0.0],    # High ester
    '30_05':[30.0,0.0,5.0,0.0],     # Low both
    '40_22':[40.0,0.0,22.0,0.0],    # Low ester
    '40_100':[40.0,0.0,100,0.0],     # High apa
    '30_30':[30.0, 0.0, 30.0, 0.0], # Same concentration
    '80_05':[80.0, 0.0, 5.0, 0.0]   # High ester Low apa
}

In [6]:
def model_output(params):
    kcat1, kcat2, Km1, Km2, Tmax, Ken, kAB, kAN, kAOH, kNH, CAB0, CAN0, CNH0, CAOH0 = params
    initial_state = [CAB0, CAN0, CNH0, CAOH0]
    sol = ode15s_amox(params[:-4],initial_state,t_eval)
    return sol  # Transpose to get time points as rows and variables as columns


In [7]:
def calculate_bounds(P, factor=2.6):
    bounds = []
    for i, param in enumerate(P):
        if i == 4:  # Tmax (special case)
            bounds.append([0, 1])
        else:
            lower_bound = param * (1 - factor)
            upper_bound = param * (1 + factor)
            if lower_bound < 0:
                lower_bound = 0.01
            bounds.append([lower_bound, upper_bound])

    return bounds

In [10]:

# Calculate bounds for the parameters in P
param_bounds = calculate_bounds(P)
print(param_bounds)

[[0.01, 0.6408], [0.01, 1.1772], [0.01, 28.458000000000002], [0.01, 45.0324], [0, 1], [0.01, 51.66], [0.01, 13.607999999999999], [0.01, 33.0264], [0.01, 39.2652], [0.01, 223.3584]]


In [11]:
# Define the problem
problem = {
    'num_vars': 10,
    'names': [
        'kcat1',
        'kcat2',
        'Km1',
        'Km2',  
        'Tmax', 
        'Ken',  
        'kAB',  
        'kAN',  
        'kAOH', 
        'kNH'],
     'bounds': np.array(param_bounds)
}
print(problem['bounds'].shape)

(10, 2)


In [14]:
# Generate samples using Sobol sequence
param_values = sobol.sample(problem, 2048)
t_eval = np.linspace(0, 500, 201)
print(t_eval)
print("N samples: ",param_values.shape)

[  0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90.   92.5  95.   97.5 100.  102.5 105.  107.5 110.  112.5 115.  117.5
 120.  122.5 125.  127.5 130.  132.5 135.  137.5 140.  142.5 145.  147.5
 150.  152.5 155.  157.5 160.  162.5 165.  167.5 170.  172.5 175.  177.5
 180.  182.5 185.  187.5 190.  192.5 195.  197.5 200.  202.5 205.  207.5
 210.  212.5 215.  217.5 220.  222.5 225.  227.5 230.  232.5 235.  237.5
 240.  242.5 245.  247.5 250.  252.5 255.  257.5 260.  262.5 265.  267.5
 270.  272.5 275.  277.5 280.  282.5 285.  287.5 290.  292.5 295.  297.5
 300.  302.5 305.  307.5 310.  312.5 315.  317.5 320.  322.5 325.  327.5
 330.  332.5 335.  337.5 340.  342.5 345.  347.5 350.  352.5 355.  357.5
 360.  362.5 365.  367.5 370.  372.5 375.  377.5 380.  382.5 385.  387.5
 390.  392.5 395.  397.5 400.  402.5 405.  407.5 41

In [47]:
import datetime
num_cores = 6  # Use all available cores
results = {}
for name,ic in inital_conditions.items():
    print("Starting...",name,"\t Time: ",datetime.datetime.now().time())
    results[name] = Parallel(n_jobs=num_cores)(delayed(model_output)(np.concatenate((params,ic))) for params in param_values)
    print(f"Ending {name}","\t Time: ",print(datetime.datetime.now().time()))


Starting... 80_60 	 Time:  13:33:30.692476


NameError: name 'enzymic_amox' is not defined

In [ ]:
results_clean = {}
for name,result in results.items():
    result_clean = []
    for result_array in result:
        if result_array.shape[0] == 201:
            result_clean.append(result)
        else:
            result_clean.append(np.zeros((201,4)))
    results_clean[name] = result_clean
    

In [ ]:
import pickle

# Store the Si_list to a file
with open('result_raw.pkl','wb') as f:
    pickle.dump(results, f)
with open('result_clean.pkl', 'wb') as f:
    pickle.dump(results_clean, f)

In [ ]:
from SALib.analyze import sobol  # Correctly import analyze
import numpy as np

all_si = {}
for name, results in result_clean.items():
    Y = np.array(results)
    print("Starting...",name,"\t Time: ",datetime.datetime.now().time())

    print(f"Shape of Y: {Y.shape}")
    Si_list = []
    for i in range(Y.shape[1]):  # Iterate over time points
        for j in range(Y.shape[2]):  # Iterate over each output variable (4 outputs)
            print(f"Processing time point {i}/{Y.shape[1]}, output {j}/{Y.shape[2]}", end='\r')

            # Perform Sobol analysis on the output variable j at time point i
            Si = sobol.analyze(problem, Y[:, i, j], calc_second_order=True, print_to_console=False)
            
            # Store the result for each time point and output variable
            Si_list.append(Si)

            # Optionally print the first-order sensitivity indices for debugging
            print(f"S1 at time point {i}, output {j}: {Si['S1']}")
    all_si[name] = Si_list
    print(f"Ending {name}","\t Time: ",print(datetime.datetime.now().time()))


In [ ]:
with open('all_si_luci.pkl', 'wb') as f:
    pickle.dump(all_si, f)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plots_folder = "sobol_plots"
os.makedirs(plots_folder, exist_ok=True)
outputs = ['CAB', 'CAN', 'CNH', 'CAOH']

for name, Si_list in all_si.items():
    Y = np.array(results_clean[name])
    n_timepoints = Y.shape[1]
    n_outputs = Y.shape[2]
    output_dir = f"sobol_plots/{name}"
    os.makedirs(output_dir,exist_ok=True)

    # Generate and save the plots for each output

    for j in range(n_outputs):  # Loop over each output variable
        S1_values = np.array([Si_list[i]['S1'] for i in range(j, len(Si_list), n_outputs)])
        ST_values = np.array([Si_list[i]['ST'] for i in range(j, len(Si_list), n_outputs)])
        S2_values = np.array([Si_list[i]['S2'] for i in range(j, len(Si_list), n_outputs)])  # Second-order indices

        # Plot S1 (First-order sensitivity indices) for each parameter across time points
        plt.figure(figsize=(12, 6))
        for k in range(S1_values.shape[1]):  # Loop over parameters
            plt.plot(range(n_timepoints), S1_values[:, k], label=f'S1 - {problem["names"][k]}')
        plt.title(f'First-order Sobol indices (S1) for {outputs[j]}')
        plt.xlabel('Time Point')
        plt.ylabel('Sobol Index (S1)')
        plt.legend(loc='best')
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, f'{name}_{outputs[j]}_S1.png'))
        plt.close()

        # Plot ST (Total-order sensitivity indices) for each parameter across time points
        plt.figure(figsize=(12, 6))
        for k in range(ST_values.shape[1]):  # Loop over parameters
            plt.plot(range(n_timepoints), ST_values[:, k], label=f'ST - {problem["names"][k]}')
        plt.title(f'Total-order Sobol indices (ST) for {outputs[j]}')
        plt.xlabel('Time Point')
        plt.ylabel('Sobol Index (ST)')
        plt.legend(loc='best')
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, f'{name}_{outputs[j]}_ST.png'))
        plt.close()

        # Aggregate S2 (Second-order indices) across time points into a heatmap
        mean_S2 = np.mean(S2_values, axis=0)  # Mean across time points for each pair (n_params x n_params)
        
        # Plot a heatmap of the mean second-order Sobol indices
        plt.figure(figsize=(10, 8))
        sns.heatmap(mean_S2, annot=True, cmap='viridis', xticklabels=problem['names'], yticklabels=problem['names'])
        plt.title(f'Heatmap of Mean Second-order Sobol Indices (S2) for {outputs[j]}')
        plt.savefig(os.path.join(output_dir, f'{name}_{outputs[j]}_S2_heatmap.png'))
        plt.close()

    print(f"Plots and heatmaps saved in: {output_dir}")
